In [ ]:
import os,shutil
from keras import models,layers

original_data_dir=r'F:\work\pywork\ipython\keras_data\dog&cat\train'
base_dir=r'F:\work\pywork\ipython\keras_data\dog&cat_small'

train_dir=os.path.join(base_dir,'train')
val_dir=os.path.join(base_dir,'val')
test_dir=os.path.join(base_dir,'test')


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
def plt_loss_acc(history):
    acc=history.history['acc']
    val_acc=history.history['val_acc']
    loss=history.history['loss']
    val_loss=history.history['val_loss']



    epochs=range(1,len(acc)+1)
    plt.plot(epochs,acc,'bo',label='train_acc')
    plt.plot(epochs,val_acc,'b--',label='val_acc')

    plt.legend()
    plt.figure()
    plt.plot(epochs,loss,'go',label='train_loss')
    plt.plot(epochs,val_loss,'g--',label='val_loss')
    plt.legend()

In [ ]:
from keras.applications import VGG16
conv_base=VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(150,150,3))


In [ ]:
model=models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

In [ ]:
model.trainable=True

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers,losses

train_datagen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
test_datagen=ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='binary')

val_generator=test_datagen.flow_from_directory(
    val_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='binary')


In [ ]:
conv_base.trainable=True

In [ ]:
len(model.trainable_weights)

In [ ]:
set_trainable=False
for layer in conv_base.layers:
    if layer.name=='block5_conv1':
        set_trainable=True
    if set_trainable:
        layer.trainable=True
    else:
        layer.trainable=False
        

In [ ]:
len(model.trainable_weights)

In [ ]:
model.compile(optimizer=optimizers.RMSprop(lr=1e-5),
             loss=losses.binary_crossentropy,
             metrics=['acc'])

In [ ]:
history=model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        epochs=100,
        validation_data=val_generator,
        validation_steps=20)